In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import pandas as pd
import re
import nltk
import string

!pip install vaderSentiment

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.5 MB/s eta 0:00:00


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
df = pd.read_excel("the_ordinary_reviews_final02.xlsx")
print(f"Total Records: {len(df)}")
print("Missing Values:\n", df.isnull().sum())

Total Records: 351
Missing Values:
 id                  0
reviewer_name       0
review_text         0
platform            0
likes               0
timestamp           0
discovery_source    0
dtype: int64


In [4]:
duplicate_count = df.duplicated(subset=['review_text']).sum()
print(f"Total Duplicate Reviews: {duplicate_count}")

Total Duplicate Reviews: 7


In [5]:
df = df.drop_duplicates(subset=['review_text'])
print(f"Total Records after removal of duplicates: {len(df)}")

Total Records after removal of duplicates: 344


In [9]:
df = df[df['review_text'].str.len() > 2]

In [ ]:
df.to_excel('/cleaned_reviews.xlsx', index=False)
print("Dataset is cleaned and saved as'cleaned_reviews.xlsx'")

Dataset is cleaned and saved as'cleaned_reviews.xlsx'


In [ ]:

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

df = pd.read_excel("cleaned_reviews.xlsx")

analyzer = SentimentIntensityAnalyzer()


def preprocess_phrases(text):
    text = str(text).lower()
    text = text.replace("game changer for me", "game_changer_for_me")
    text = text.replace("total game changer", "total_game_changer")
    text = text.replace("go to for me", "go_to_for_me")
    text = text.replace("go-to", "go_to")
    text = text.replace("go to", "go_to")
    text = text.replace("no side effects", "no_side_effects")
    text = text.replace("blown away", "blown_away")
    text = text.replace("not in love", "not_in_love")
    text = text.replace("not sure", "not_sure")
    text = text.replace("not convinced by the hype", "not_convinced_by_the_hype")
    text = text.replace("still unsure", "still_unsure")
    text = text.replace("no glow", "no_glow")
    text = text.replace("didn’t cause any issues", "no_issues")
    text = text.replace("kind of average", "kind_of_average")
    text = text.replace("not amazing", "not_amazing")
    text = text.replace("not terrible", "not_terrible")
    text = text.replace("too heavy", "too_heavy")
    text = text.replace("a bit overpriced", "overpriced")
    return text

# Custom lexicon update (underscore versions for preprocessed phrases)
analyzer.lexicon.update({
    "game_changer_for_me": 4.0,
    "go_to": 4.0,
    "go_to_for_me": 4.0,
    "no_side_effects": 1.0,
    "amazing": 4.0,
    "broke": -3.0,
    "blown_away": 3.0,
    "absorbs": 3.0,
    "okay": 0,
    "not_in_love": 0,
    "calmer": 3.0,
    "helped": 3.0,
    "not_sure": 0,
    "average": 0,
    "total_game_changer": 4.0,
     "no_side_effects": 1.0,
    "no_glow": -1.5,
    "still_unsure": 0,
    "no_issues": 1.0 ,
     "not_sure": 0,
    "not_amazing": 0,
    "not_terrible": 0,
    "kind_of_average": 0,
    "too_heavy" : -2.0,
    "overpriced": -2.0,
    "disappointed": -2.5,
    "expected more": -1.0

})


df['processed_text'] = df['review_text'].apply(preprocess_phrases)

def get_sentiment_scores(text):
    scores = analyzer.polarity_scores(str(text))
    compound = scores['compound']
    if compound >= 0.05:
        sentiment = 'Positive'
    elif compound <= -0.05:
        sentiment = 'Negative'
    else:
        sentiment = 'Neutral'
    return pd.Series([compound, sentiment])

df[['vader_polarity', 'vader_label']] = df['processed_text'].apply(get_sentiment_scores)

df.to_excel("vader_custom_scored_reviews.xlsx", index=False)
print("Cleaned Dataset is VADER classified and saved as'vader_custom_scored_reviews.xlsx'")
df['vader_label'].value_counts()

Cleaned Dataset is VADER classified and saved as'vader_custom_scored_reviews.xlsx'


,count
vader_label,
Positive,252
Negative,50
Neutral,41


In [ ]:
#MODEL DEVELOPMENT

df = pd.read_excel("training_set.xlsx")
df = df[['review_text', 'verified_label']].dropna()

In [22]:

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = str(text).lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['cleaned_text'] = df['review_text'].apply(preprocess_text)


In [14]:
label_map = {'Positive': 1, 'Neutral': 0, 'Negative': -1}
df['label'] = df['verified_label'].map(label_map)

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer


X_raw = df['cleaned_text']
y     = df['label']
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X_raw, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_train = vectorizer.fit_transform(X_train_raw)
X_test  = vectorizer.transform(X_test_raw)



In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score

nb_model  = MultinomialNB()
svm_model = LinearSVC(max_iter=5000)
nb_model.fit(X_train, y_train)
svm_model.fit(X_train, y_train)

df_model = pd.read_excel("training_set.xlsx")
df_model = df_model[['review_text', 'verified_label']].dropna()

df_vader = pd.read_excel("vader_custom_scored_reviews.xlsx")
df_vader = df_vader[['review_text', 'vader_label']]

df = pd.merge(df_model, df_vader, on='review_text', how='inner')
accuracy = accuracy_score(df['verified_label'], df['vader_label'])
print(f"VADER Accuracy: {accuracy * 100:.2f}%")

print("\n🔍 Detailed Performance Metrics:\n")
print(classification_report(df['verified_label'], df['vader_label'], digits=3))

for name, model in [('Naïve Bayes', nb_model), ('SVM', svm_model)]:
    preds = model.predict(X_test)
    print(f"{name} Accuracy: {accuracy_score(y_test, preds) * 100:.2f}%")
    print(classification_report(
        y_test, preds,
        target_names=['Negative','Neutral','Positive']
    ))



VADER Accuracy: 72.07%

🔍 Detailed Performance Metrics:

              precision    recall  f1-score   support

    Negative      0.735     0.735     0.735        34
     Neutral      0.800     0.541     0.645        37
    Positive      0.673     0.875     0.761        40

    accuracy                          0.721       111
   macro avg      0.736     0.717     0.714       111
weighted avg      0.734     0.721     0.714       111

Naïve Bayes Accuracy: 60.87%
              precision    recall  f1-score   support

    Negative       0.60      0.43      0.50         7
     Neutral       0.80      0.50      0.62         8
    Positive       0.54      0.88      0.67         8

    accuracy                           0.61        23
   macro avg       0.65      0.60      0.59        23
weighted avg       0.65      0.61      0.60        23

SVM Accuracy: 43.48%
              precision    recall  f1-score   support

    Negative       0.38      0.43      0.40         7
     Neutral       0.4

In [17]:
!pip install -q streamlit pyngrok openpyxl scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.7 MB/s eta 0:00:00


In [ ]:
with open("app.py", "w") as f:
    f.write('''\

import streamlit as st
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

tab1, tab2 = st.tabs(["Sentiment Analysis On Social Media Reviews on the brand, The Ordinary.", "Model Comparison"])

with tab1:
    st.markdown("<h1 style='text-align: center; color: #B9D9EB; font-family: monospace;'>Sentiment Analysis Dashboard</h1>", unsafe_allow_html=True)
    df = pd.read_excel("vader_custom_scored_reviews.xlsx")

    df['vader_label'] = df['vader_label'].str.lower()

    sentiment_counts = df['vader_label'].value_counts()
    labels = sentiment_counts.index.tolist()

    fig = px.pie(
        names=labels,
        values=sentiment_counts.values,
        color=labels,
        color_discrete_map={
            'positive': '#ffabab',
            'neutral': '#0068c9',
            'negative': '#83c8ff'
        },
        title="Distribution of Sentiments in Reviews"
    )

    fig.update_traces(textinfo='percent+label')
    fig.update_layout(title_font_size=20)

    st.plotly_chart(fig)
    st.markdown("---")
    st.markdown("### Conclusion")
    st.write("""
    The sentiment analysis reveals a predominantly positive perception of the brand, with 73.5% of customer reviews expressing positive sentiment.
    While negative reviews account for 14.6% and neutral reviews 12.0%, the overwhelming positive feedback indicates **strong overall customer satisfaction.**

    Addressing the concerns raised in the negative reviews remains important for continuous improvement.
    """)
    st.markdown("---")




    # Platform vs sentiment
    platform_sentiment = df.groupby(['platform', 'vader_label']).size().unstack().fillna(0)
    st.write("### Sentiment Distribution by Platform")
    st.bar_chart(platform_sentiment)

    st.markdown("### Conclusion")
    st.write("""
    Across Facebook, Instagram, and Twitter, the sentiment towards the skincare brand shows a consistent pattern with the highest volume of positive sentiment,
    followed by neutral, and then negative sentiment having the lowest volume.
    Facebook and Twitter show a similar overall volume of mentions, while Instagram has a notably lower volume across all sentiment categories.
    """)
    st.markdown("---")




    # Time vs sentiment
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
    df = df.dropna(subset=['timestamp'])

    df['month'] = df['timestamp'].dt.to_period('M').dt.to_timestamp()

    monthly_sentiment = df.groupby(['month', 'vader_label']).size().unstack().fillna(0)

    st.write("### Sentiment Over Time")
    st.line_chart(monthly_sentiment)
    st.markdown("### Conclusion")
    st.write("""
    The sentiment towards the skincare brand from **2018 to 2024** has been predominantly **positive**, with frequent noticeable peaks.
    **Negative sentiment** remains consistently low, showing only occasional, mild spikes.
    Meanwhile, **neutral sentiment** fluctuates modestly over time.
    Overall, this indicates a generally **favorable public perception** of the brand, with periods of increased engagement likely linked to campaigns, events, or product launches.
    """)
    st.markdown("---")





    # Discovery source vs sentiment
    discovery_sentiment = df.groupby(['discovery_source', 'vader_label']).size().unstack().fillna(0)
    discovery_sentiment = discovery_sentiment.loc[discovery_sentiment.sum(axis=1).sort_values(ascending=False).index]

    st.markdown("### Sentiment by Discovery Source")
    st.bar_chart(discovery_sentiment)

    st.markdown("### Conclusion")
    st.write("""
    Among all discovery sources, **TikTok**, **Search Engines**, and **Influencers** brought in the highest number of total reviews,
    with **TikTok** and **Influencers** showing a strong skew toward **positive sentiment**.

    Interestingly, sources like **Friends** and **In-store experiences** also led to a large number of reviews, but the sentiment distribution there was more mixed.

    This suggests that **social media platforms and influencer marketing** not only drive high engagement but also tend to result in
    **more favorable customer perceptions**.
    """)
    st.markdown("---")




    # Review Length by Sentiment
    st.markdown("### Review Length by Sentiment")
    df['review_length'] = df['processed_text'].apply(lambda x: len(str(x).split()))

    fig1 = px.box(df,
                  x='vader_label',
                  y='review_length',
                  color='vader_label',
                  color_discrete_map={
                      "Positive": "#8dd3c7",
                      "Neutral": "#ffffb3",
                      "Negative": "#fb8072"
                  },
                  title="How Review Length Varies with Sentiment")

    fig1.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        font_color='white',
        title_font_size=18
    )
    st.plotly_chart(fig1, use_container_width=True)
    st.markdown("### Conclusion")
    st.write("""
    Positive reviews tend to be longer and receive more likes, suggesting that detailed, enthusiastic feedback resonates well with others.
    Negative reviews, while shorter on average, also garner attention, possibly due to their emotional impact.
    Neutral reviews are typically brief and receive the least engagement.
    """)
    st.markdown("---")




    # Likes by Sentiment
    st.markdown("### Likes by Sentiment")

    fig2 = px.box(df,
                  x='vader_label',
                  y='likes',
                  color='vader_label',
                  color_discrete_map={
                      "Positive": "#8dd3c7",
                      "Neutral": "#ffffb3",
                      "Negative": "#fb8072"
                  },
                  title="How Likes Vary with Sentiment")

    fig2.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        font_color='white',
        title_font_size=18
    )
    st.plotly_chart(fig2, use_container_width=True)
    st.markdown("### Conclusion")
    st.write("""
    Content with strong sentiment (positive or negative) is generally more engaging and tends to receive more likes than neutral content.
    This reinforces the idea that emotional tone plays a significant role in social media engagement.
    """)
    st.markdown("---")



    st.markdown("### Most Liked Reviews by Sentiment")

    df = df.dropna(subset=['review_text', 'vader_label', 'likes'])
    df['likes'] = pd.to_numeric(df['likes'], errors='coerce')
    df = df.dropna(subset=['likes'])
    df['vader_label'] = df['vader_label'].str.capitalize()

    for sentiment in ['Positive', 'Neutral', 'Negative']:
        st.write(f"#### {sentiment} Reviews")
        top_reviews = df[df['vader_label'] == sentiment].sort_values(by='likes', ascending=False).head(3)
        st.dataframe(top_reviews[['review_text', 'likes', 'platform']])


    st.markdown("### Conclusion")
    st.write("""The most liked reviews across all sentiments are detailed and emotionally expressive.
    Positive reviews highlight product effectiveness, while top negative reviews are often tied to skin reactions.
      This suggests users resonate most with personal experiences that feel relatable or extreme.""")
    st.markdown("---")




    st.markdown("### Average Sentiment Score by Discovery Source")
    source_sentiment = df.groupby('discovery_source')['vader_polarity'].mean().sort_values(ascending=False).reset_index()

    fig = px.bar(
        source_sentiment,
        x='discovery_source',
        y='vader_polarity',
        title="Average Sentiment Score by Discovery Source",
        labels={'vader_polarity': 'Avg Sentiment Score', 'discovery_source': 'Discovery Source'},
        color='vader_polarity',
        color_continuous_scale='RdYlGn'
    )

    fig.update_layout(xaxis_tickangle=-45)
    st.plotly_chart(fig)
    st.markdown("### Conclusion")
    st.write("""Marketers and brands might want to focus more on Social Media, Reddit, and Search Engines to improve perception and engagement.
    Beauty Blogs and Ads may need reassessment or different strategies to improve their public perception.
    """)




with tab2:
    st.markdown("### Model Accuracy Comparison")
    st.write("This section compares the performance of different sentiment classification models:")

    # Create a small DataFrame of your accuracy scores
    accuracy_df = pd.DataFrame({
        "Model": ["VADER", "Naive Bayes", "SVM"],
        "Accuracy": [71.17, 60.87, 43.48]
    })

    # Build the Plotly bar chart
    fig_acc = px.bar(
        accuracy_df,
        x="Model",
        y="Accuracy",
        text="Accuracy",
        range_y=[0, 100],
        color="Model",
        color_discrete_map={"VADER": "#636EFA", "Naive Bayes": "#EF553B", "SVM": "#00CC96"}
    )

    fig_acc.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
    fig_acc.update_layout(
        yaxis_title="Accuracy (%)",
        xaxis_title="Model",
        title="Model Accuracy Comparison",
        showlegend=False,
        uniformtext_minsize=8,
        uniformtext_mode='hide'
    )

    st.plotly_chart(fig_acc, use_container_width=True)

    st.markdown("### Detailed Metrics")

    vader_metrics = """
| Class     | Precision | Recall | F1-score | Support |
|-----------|-----------|--------|----------|---------|
| Negative  | 0.735     | 0.735  | 0.735    | 34      |
| Neutral   | 0.826     | 0.514  | 0.633    | 37      |
| Positive  | 0.648     | 0.875  | 0.745    | 40      |
| **Avg (W)**| 0.734    | 0.712  | 0.705    | 111     |
    """

    nb_metrics = """
| Class     | Precision | Recall | F1-score | Support |
|-----------|-----------|--------|----------|---------|
| Negative  | 0.60      | 0.43   | 0.50     | 7       |
| Neutral   | 0.80      | 0.50   | 0.62     | 8       |
| Positive  | 0.54      | 0.88   | 0.67     | 8       |
| **Avg (W)**| 0.65     | 0.61   | 0.60     | 23      |
    """

    svm_metrics = """
| Class     | Precision | Recall | F1-score | Support |
|-----------|-----------|--------|----------|---------|
| Negative  | 0.38      | 0.43   | 0.40     | 7       |
| Neutral   | 0.43      | 0.38   | 0.40     | 8       |
| Positive  | 0.50      | 0.50   | 0.50     | 8       |
| **Avg (W)**| 0.44     | 0.43   | 0.43     | 23      |
    """

    st.markdown("#### VADER")
    st.markdown(vader_metrics)

    st.markdown("#### Naive Bayes")
    st.markdown(nb_metrics)

    st.markdown("#### SVM")
    st.markdown(svm_metrics)

    st.markdown("---")
    st.markdown("### Summary")
    st.write("""
VADER performed best overall, especially with Positive and Negative reviews, thanks to its sentiment-focused lexicon.
Naive Bayes had good recall on Positive but struggled with Negative.
SVM underperformed on this dataset, likely due to the size of the dataset.
    """)








''')


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🌐 Streamlit app is live at: NgrokTunnel: "https://2650-34-73-190-253.ngrok-free.app" -> "http://localhost:8501"


In [34]:
with open("requirements.txt", "w") as f:
    f.write("""\
streamlit
pyngrok
pandas
nltk
matplotlib
seaborn
plotly
scikit-learn
openpyxl
vaderSentiment
""")

from google.colab import files
files.download("requirements.txt")
files.download("app.py")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

['.config',
 'requirements.txt',
 'svm_model.pkl',
 'app.py',
 'drive',
 'vectorizer.pkl',
 'vader_custom_scored_reviews.xlsx',
 'naive_bayes_model.pkl',
 'sample_data']